In [1]:
import pandas as pd
import json
from arcgis.gis import GIS

In [2]:
# convert nested json file into pandas dataframe
input_json = "mo_charity_data.json"
with open(input_json, "r") as f:
    data = json.load(f)
df = pd.json_normalize(data)


In [4]:
# trim table to subset of columns
columns = [
    "charityName", "ein", "mission", "websiteURL", "currentRating.score", "currentRating.rating", "currentRating.financialRating.score", 
    "currentRating.financialRating.rating","currentRating.accountabilityRating.score", "currentRating.accountabilityRating.rating", 
    "category.categoryName", "cause.causeName", "irsClassification.deductibility", "irsClassification.subsection", 
    "mailingAddress.streetAddress1", "mailingAddress.streetAddress2", "mailingAddress.city", "mailingAddress.stateOrProvince", 
    "mailingAddress.postalCode"
    ]

df = df[columns]

In [5]:
# rename all columns
newColumns = [
    "charityName", "ein", "mission", "websiteURL", "currentRatingScore", "currentRating", "financialRatingScore", "financialRating",
    "accountabilityRatingscore", "accountabilityRating", "categoryName", "causeName", "deductibility", "subsection", "address1", 
    "address2", "city", "state", "zip"
    ]

df.set_axis(newColumns, axis=1, inplace=True)

In [9]:
# add additional blank columns
df = df.assign(needPrio1="", needPrio2="", needPrio3="", needPrio4="", needPrio5="")

In [11]:
# geocode addresses and return data as a JSON string
gis = GIS("Pro")
address_fields ={
    "Address": "address1",
    "City": "city",
    "State": "state",
    "ZIP": "zip"
}
fc = gis.content.import_data(df, address_fields)
fc_dict = dict(fc.properties)
jsonString = json.dumps({"featureCollection": {"layers": [fc_dict]}})

In [15]:
# Create portal item from JSON string
item_properties = {
    "title": "hackathonTest",
    "description": "Charities in Missouri rated by Charity Navigator",
    "tags": "Hackathon",
    "text": jsonString,
    "type": "Feature Collection"
}

item = gis.content.add(item_properties)

In [16]:
df.head()

,charityName,ein,mission,websiteURL,currentRatingScore,currentRating,financialRatingScore,financialRating,accountabilityRatingscore,accountabilityRating,...,address1,address2,city,state,zip,needPrio1,needPrio2,needPrio3,needPrio4,needPrio5
0,H.E.R.O.E.S. Care,010777850,H.E.R.O.E.S. Care is a collaborative effort am...,https://heroescare.org/,96.46,4,95.00,4,100,4,...,330 Sun Valley Circle,None,Fenton,MO,63049,,,,,
1,Camp Fire National Headquarters,131623921,"In a world where so many feel disconnected, Ca...",http://campfire.org/,79.26,2,70.67,2,100,4,...,1801 Main Street,Suite 200,Kansas City,MO,64106,,,,,
2,"National Council of the United States, Society...",135562362,"Inspired by Gospel values, the Society of St. ...",http://www.svdpusa.org,86.77,3,88.84,3,85,3,...,58 Progress Parkway,None,Maryland Heights,MO,63043,,,,,
3,Little Bit Foundation,200126713,"Established in 2001, The Little Bit Foundation...",https://thelittlebitfoundation.org/,85.86,3,93.77,4,81,3,...,516 Hanley Industrial Court,None,St. Louis,MO,63144,,,,,
4,Meds & Food for Kids,201257910,Meds & Food for Kids (MFK) is dedicated to sav...,https://mfkhaiti.org/,95.77,4,94.02,4,100,4,...,4488 Forest Park,Suite 230,St. Louis,MO,63108,,,,,


In [17]:
df["ein"]

0      010777850
1      131623921
2      135562362
3      200126713
4      201257910
         ...    
162    680051386
163    742148471
164    816079539
165    831966879
166    952630437
Name: ein, Length: 167, dtype: object

In [18]:
fc.properties


{
  "layerDefinition": {
    "currentVersion": 10.81,
    "id": 0,
    "name": "",
    "type": "Feature Layer",
    "displayField": "",
    "description": "",
    "copyrightText": "",
    "defaultVisibility": true,
    "relationships": [],
    "isDataVersioned": false,
    "supportsAppend": true,
    "supportsCalculate": true,
    "supportsASyncCalculate": true,
    "supportsTruncate": false,
    "supportsAttachmentsByUploadId": true,
    "supportsAttachmentsResizing": true,
    "supportsRollbackOnFailureParameter": true,
    "supportsStatistics": true,
    "supportsExceedsLimitStatistics": true,
    "supportsAdvancedQueries": true,
    "supportsValidateSql": true,
    "supportsCoordinatesQuantization": true,
    "supportsLayerOverrides": true,
    "supportsTilesAndBasicQueriesMode": true,
    "supportsFieldDescriptionProperty": true,
    "supportsQuantizationEditMode": true,
    "supportsApplyEditsWithGlobalIds": false,
    "supportsReturningQueryGeometry": true,
    "advancedQueryCap